# Random fields

In [51]:
#pip install python-crfsuite
import json
import re
from sklearn.model_selection import train_test_split
import pycrfsuite

In [52]:
def clean_word(word):
    word = word.replace(' ','')
    word = word.replace('.','')
    return(word)

def rm_mltp(text):
    text = re.sub(' +', ' ',text)
    return(text)

In [78]:
def alternative_data_base(data):
    new_data = []
    for element in data:
        for key,value in element.items():
            list_temp = []
            for dico in value:
                text = rm_mltp(dico['text'])
                for word in text.split(' '):
                    if word == ' ':
                        print(text)
                    cleaned_word = clean_word(word)
                    if cleaned_word != '':
                        unite_dico = {'text' : cleaned_word}
                        if 'entity' in dico.keys():
                            unite_dico['entity'] = dico['entity']
                        else :
                            unite_dico['entity'] = 'None'
                        list_temp.append(unite_dico)
        new_data.append(list_temp)
    return(new_data)

In [79]:
number = 0
list_query_type = ['AddToPlaylist','GetWeather','SearchCreativeWork' , 'BookRestaurant','PlayMusic','RateBook' ,'SearchScreeningEvent']
type_of_query = list_query_type[number]
data = json.load(open(type_of_query+ '.json'))[type_of_query]
data = alternative_data_base(data)

In [86]:
#Rajouter une fonction pour 
def word2features(query, i):
    word = query[i]['text']
    #postag = query[i]['entity']

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'place_in_query=%s' % str(i),
        'len_query=%s' % str(len(query))
        #,'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a query
    if i > 0:
        word1 = query[i-1]['text']
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a query'
        features.append('BOS')

    # Features for words that are not
    # at the end of a queryument
    if i < len(query)-1:
        word1 = query[i+1]['text']
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isdigit=%s' % word1.isdigit()
            #,'+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a query'
        features.append('EOS')

    return features

In [87]:
# A function for extracting features in queries
def extract_features(query):
    return [word2features(query, i) for i in range(len(query))]

# A function fo generating the list of labels for each query
def get_entities(query):
    return [word['entity'] for word in query]

X = [extract_features(query) for query in data]
y = [get_entities(query) for query in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [88]:
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 10546
Seconds required: 0.069

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 22248.357471
Feature norm: 1.000000
Error norm: 8569.053112
Active features: 10478
Line search trials: 1
Line search step: 0.000077
Seconds required for this iteration: 0.021

***** Iteration #2 *****
Loss: 20801.503294
Feature norm: 0.980307
Error norm: 5969.314776
Active features: 10330
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.012

***** Iteration #3 *****
Loss: 19746.414695
Feature norm: 1.058981
Error norm: 4817.590545
Active features: 10418
Line search trials: 1
Line search step: 1.000000
Seconds required for th

***** Iteration #41 *****
Loss: 671.734863
Feature norm: 84.206345
Error norm: 41.123926
Active features: 5312
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.019

***** Iteration #42 *****
Loss: 663.116065
Feature norm: 84.421889
Error norm: 54.599553
Active features: 5166
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.017

***** Iteration #43 *****
Loss: 654.622933
Feature norm: 84.889419
Error norm: 23.537541
Active features: 5038
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.017

***** Iteration #44 *****
Loss: 654.393986
Feature norm: 86.156474
Error norm: 321.194550
Active features: 4884
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.016

***** Iteration #45 *****
Loss: 641.727327
Feature norm: 86.417573
Error norm: 32.072110
Active features: 4909
Line search trials: 1
Line search step: 1.000000
Seconds required for th

***** Iteration #87 *****
Loss: 579.776778
Feature norm: 92.626024
Error norm: 12.731058
Active features: 3246
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.016

***** Iteration #88 *****
Loss: 579.298400
Feature norm: 92.614908
Error norm: 14.007506
Active features: 3244
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.017

***** Iteration #89 *****
Loss: 578.866933
Feature norm: 92.626823
Error norm: 18.232756
Active features: 3243
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.013

***** Iteration #90 *****
Loss: 578.530657
Feature norm: 92.653193
Error norm: 11.942916
Active features: 3241
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.019

***** Iteration #91 *****
Loss: 578.262021
Feature norm: 92.676961
Error norm: 15.465147
Active features: 3246
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

***** Iteration #138 *****
Loss: 572.036344
Feature norm: 94.498649
Error norm: 9.828539
Active features: 3014
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.013

***** Iteration #139 *****
Loss: 572.003911
Feature norm: 94.541167
Error norm: 18.322309
Active features: 3015
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.015

***** Iteration #140 *****
Loss: 571.897188
Feature norm: 94.556898
Error norm: 13.337119
Active features: 3011
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.015

***** Iteration #141 *****
Loss: 571.842835
Feature norm: 94.597712
Error norm: 14.407941
Active features: 3008
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.012

***** Iteration #142 *****
Loss: 571.761257
Feature norm: 94.612583
Error norm: 12.764909
Active features: 3003
Line search trials: 1
Line search step: 1.000000
Seconds required for

***** Iteration #191 *****
Loss: 569.534356
Feature norm: 94.917480
Error norm: 8.201878
Active features: 2947
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.014

***** Iteration #192 *****
Loss: 569.511695
Feature norm: 94.916759
Error norm: 8.839458
Active features: 2943
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.013

***** Iteration #193 *****
Loss: 569.491076
Feature norm: 94.919584
Error norm: 9.670649
Active features: 2944
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.014

***** Iteration #194 *****
Loss: 569.470556
Feature norm: 94.919883
Error norm: 10.307254
Active features: 2946
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.012

***** Iteration #195 *****
Loss: 569.437928
Feature norm: 94.921946
Error norm: 7.355517
Active features: 2941
Line search trials: 1
Line search step: 1.000000
Seconds required for th

In [89]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 10
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

i (None)
want (None)
to (None)
add (None)
an (None)
album (music_item)
to (None)
noreen's (playlist_owner)
endorphin (playlist)
rush (playlist)
playlist (None)
